In [31]:
from optparse import OptionParser
from collections import OrderedDict
import pickle
import os
import glob
import sys
import threading
import time

import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import h5py
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf

sys.path.append('../../basenji/')

from basenji import params
from basenji import seqnn
from basenji import vcf as bvcf

sys.path.append('../bin/')

from basenji_test import bigwig_open


UsageError: Invalid GUI request 'pdf', valid ones are:dict_keys(['nbagg', 'qt5', 'ipympl', 'qt4', 'osx', 'widget', 'gtk', 'asyncio', None, 'wx', 'inline', 'qt', 'gtk3', 'notebook', 'tk'])


In [3]:
root_dir = '/n/local/basenji'

In [4]:
params_file = os.path.join(root_dir,'manuscript/params.txt')
model_file = os.path.join(root_dir,'data/model.tf')
glob.glob(model_file+'*')

['/n/local/basenji/data/model.tf.meta',
 '/n/local/basenji/data/model.tf.index',
 '/n/local/basenji/data/model.tf.data-00000-of-00001']

In [5]:
job = params.read_job_params(params_file, require=['seq_length','num_targets'])

{'num_targets': 4229, 'link': 'exp_linear', 'cnn_filter_sizes': [22, 1, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 1], 'cnn_dilation': [1, 1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'cnn_pool': [1, 2, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1], 'batch_buffer': 4096, 'loss': 'poisson', 'seq_length': 131072, 'cnn_filters': [312, 368, 435, 514, 607, 717, 108, 108, 108, 108, 108, 108, 1365], 'batch_size': 2, 'adam_beta1': 0.97, 'learning_rate': 0.002063, 'cnn_dense': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], 'cnn_dropout': [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05], 'adam_beta2': 0.98}


In [51]:
job['cnn_filter_sizes']
job['cnn_dilation']
job['cnn_pool']
job['cnn_filters']
job['cnn_dropout']
job['num_targets']


KeyError: 'cnn_stride'

In [6]:
model = seqnn.SeqNN()

In [7]:
model.build_feed_sad(job)

INFO:tensorflow:Parsed params from job argument, and got {'cnn_dilation': [1, 1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'stop_early': False, 'num_plateau_steps': 5000, 'cnn_dense': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], 'batch_norm': True, 'ensemble_during_prediction': False, 'cnn_skip': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'learning_rate': 0.002063, 'final_l1_scale': 0.0, 'num_targets': 4229, 'batch_norm_momentum': 0.9, 'plateau_delta': 0.05, 'cnn_pool': [1, 2, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1], 'target_pool': 1, 'stop_early_num_plateau_steps': 10000, 'shift_augment_offsets': None, 'stop_early_plateau_delta': 0.03, 'batch_size': 2, 'adam_beta1': 0.97, 'learning_decay_rate': 0.2, 'nonlinearity': 'relu', 'link': 'exp_linear', 'ensemble_during_training': False, 'momentum': 0, 'target_classes': 1, 'target_sqrt': False, 'cnn_filters': [312, 368, 435, 514, 607, 717, 108, 108, 108, 108, 108, 108, 1365], 'augment_with_complement': False, 'learning_decay_steps': 200000, 'pool_by_unfolding':

In [21]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()
saver.restore(sess, model_file)

INFO:tensorflow:Restoring parameters from /n/local/basenji/data/model.tf


In [22]:
list([n.name for n in tf.get_default_graph().as_graph_def().node])

['global_step/Initializer/zeros',
 'global_step',
 'global_step/Assign',
 'global_step/read',
 'inputs',
 'targets',
 'targets_na',
 'is_training',
 'zeros',
 'stack',
 'stack_1',
 'map/TensorArray/size',
 'map/TensorArray',
 'map/TensorArray_1/size',
 'map/TensorArray_1',
 'map/TensorArrayUnstack/Shape',
 'map/TensorArrayUnstack/strided_slice/stack',
 'map/TensorArrayUnstack/strided_slice/stack_1',
 'map/TensorArrayUnstack/strided_slice/stack_2',
 'map/TensorArrayUnstack/strided_slice',
 'map/TensorArrayUnstack/range/start',
 'map/TensorArrayUnstack/range/delta',
 'map/TensorArrayUnstack/range',
 'map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3',
 'map/TensorArrayUnstack_1/Shape',
 'map/TensorArrayUnstack_1/strided_slice/stack',
 'map/TensorArrayUnstack_1/strided_slice/stack_1',
 'map/TensorArrayUnstack_1/strided_slice/stack_2',
 'map/TensorArrayUnstack_1/strided_slice',
 'map/TensorArrayUnstack_1/range/start',
 'map/TensorArrayUnstack_1/range/delta',
 'map/TensorArrayU

In [23]:
for op in tf.get_default_graph().get_operations():
    print(str(op.name))

global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
inputs
targets
targets_na
is_training
zeros
stack
stack_1
map/TensorArray/size
map/TensorArray
map/TensorArray_1/size
map/TensorArray_1
map/TensorArrayUnstack/Shape
map/TensorArrayUnstack/strided_slice/stack
map/TensorArrayUnstack/strided_slice/stack_1
map/TensorArrayUnstack/strided_slice/stack_2
map/TensorArrayUnstack/strided_slice
map/TensorArrayUnstack/range/start
map/TensorArrayUnstack/range/delta
map/TensorArrayUnstack/range
map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
map/TensorArrayUnstack_1/Shape
map/TensorArrayUnstack_1/strided_slice/stack
map/TensorArrayUnstack_1/strided_slice/stack_1
map/TensorArrayUnstack_1/strided_slice/stack_2
map/TensorArrayUnstack_1/strided_slice
map/TensorArrayUnstack_1/range/start
map/TensorArrayUnstack_1/range/delta
map/TensorArrayUnstack_1/range
map/TensorArrayUnstack_1/TensorArrayScatter/TensorArrayScatterV3
map/Const
map/TensorArray_2/size
map/TensorArr

In [24]:
tf.global_variables()

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>,
 <tf.Variable 'cnn0/conv1d/kernel:0' shape=(22, 4, 312) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/gamma:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/beta:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/moving_mean:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/moving_variance:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn1/conv1d/kernel:0' shape=(1, 312, 368) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/gamma:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/beta:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/moving_mean:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/moving_variance:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn2/conv1d/kernel:0' shape=(6, 368, 435) dtype=float32_ref>,
 <tf.Variable 'cnn2/batch_n

In [41]:
model_weights = OrderedDict([ (tensor.name, sess.run( tensor )) for tensor in tf.global_variables()[1:] ])

In [43]:
with open(os.path.join(root_dir,'manuscript/model_numpy.pkl'),'wb') as f:
    pickle.dump(model_weights, f)

In [44]:
with open(os.path.join(root_dir,'manuscript/model_numpy.pkl'),'rb') as f:
    check_weights = pickle.load(f)

In [87]:
import torch
import torch.nn as nn

In [94]:
class BasenjiConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, 
                 dense_conv=False, padding_mode='zeros', use_batchnorm=True,
                 pooling_type='max', pooling_kernel=1, pooling_stride=1,
                 activation_type='relu', dropout_prob=0.5):
        super(BasenjiConvBlock, self).__init__()
        # Set 1D convolution
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size,
                                stride, padding, dilation, groups, bias,
                                padding_mode)
        # Set Batch Norm layer
        if use_batchnorm:
            self.batch_normalization = nn.BatchNorm1d(out_channels, eps=1e-05, 
                                                      momentum=0.1, affine=True, 
                                                      track_running_stats=True)
        else:
            self.batch_normalization = Passthrough()
        # Add pooling
        if pooling_type == 'max':
            self.pooling = nn.MaxPool1d(pooling_kernel, pooling_stride)
        else:
            print('Unrecognized pooling type "{}"'.format(pooling_type), file=sys.stderr)
            exit(1)
        # Add Nonlinearity
        if activation_type == 'relu':
            self.activation = nn.ReLU()
        elif activation_type == 'exp_linear':
            self.activation = ExpLinear()
        elif activation_type == 'none':
            self.activation = Passthrough()
        else:
            print('Unrecognized activation type "{}"'.format(activation_type), file=sys.stderr)
        # Add dropout
        if dropout_prob > 0.0:
            self.dropout = Dropout1d(p=dropout_prob)
        else:
            self.dropout = Passthrough()
        # Decide if dense conv
        self.dense_conv = dense_conv
    def forward(self, input):
        hook = self.conv1d( input )
        hook = self.batch_normalization( hook )
        hook = self.pooling( hook )
        hook = self.activation( hook )
        hook = self.dropout( hook )
        if self.dense_conv:
            hook = torch.cat([input,hook],dim=1)
        return hook
    
class Dropout1d(nn.Module):
    def __init__(self, p=0.5):
        super(Dropout1d, self).__init__()
        self.dropout = nn.Dropout2d(p)
    def forward(self, input):
        return self.dropout( input.unsqueeze(-1) ).squeeze(3)

class ExpLinear(nn.Module):
    def __init__(self):
        super(ExpLinear, self).__init__()
    def forward(self, input):
        return input.clamp(min=0.) + \
               input.clamp(min=-50., max=0.).exp()

class Passthrough(nn.Module):
    def __init__(self):
        super(Passthrough, self).__init__()
    def forward(self, input):
        return input


In [100]:
class SeqNN(nn.Module):
    def __init__(self, job):
        super(SeqNN, self).__init__()
        network = []
        density = 0
        input_feat = 4
        for i in range(len(job['cnn_filter_sizes'])):
            curr_feat = job['cnn_filters'][i]
            dilation  = job['cnn_dilation'][i]
            kern_size = job['cnn_filter_sizes'][i]
            drop_prob = job['cnn_dropout'][i]
            pooling   = job['cnn_pool'][i]
            is_dense  = job['cnn_dense'][i] == 1
            if is_dense:
                density += job['cnn_filters'][i-1]
            print(
                "('cnn{}', BasenjiConvBlock({}, {}, {}, stride=1, padding=0, \n\
                                             dilation={}, groups=1, bias=False, \n\
                                             dense_conv={}, pooling_kernel={}, \n\
                                             pooling_stride={}, activation_type='relu', \n\
                                             dropout_prob={}))" \
                .format(i,input_feat, curr_feat, kern_size, dilation, is_dense, pooling, pooling, drop_prob)
            )
            network.append(
                ('cnn{}'.format(i), BasenjiConvBlock(input_feat, curr_feat, kern_size,
                                                      stride=1, padding=0, dilation=dilation, 
                                                      groups=1, bias=False, dense_conv=is_dense, 
                                                      pooling_kernel=pooling, pooling_stride=pooling, 
                                                      activation_type='relu', dropout_prob=drop_prob
                                                     ) )
            )
            input_feat = curr_feat + density


        print(
            "('final', BasenjiConvBlock({}, {}, 1, stride=1, padding=0, \n\
                                         dilation=1, groups=1, dense_conv=False, bias=True, \n\
                                         dense_conv=False, use_batchnorm=False, pooling_kernel=1, \n\
                                         pooling_stride=1, activation_type='exp_linear', \n\
                                         dropout_prob=0.0))" \
            .format(curr_feat, job['num_targets'])
        )
        network.append(
            ('final', BasenjiConvBlock(curr_feat, job['num_targets'], 1, stride=1, 
                                       padding=0, dilation=1, groups=1, bias=True, dense_conv=False,
                                       use_batchnorm=False, pooling_kernel=1, pooling_stride=1, 
                                       activation_type='exp_linear', dropout_prob=0.0
                                      ))
        )
        self.network = nn.Sequential( OrderedDict(network) )
    
    def forward(self, input):
        self.network(input)



"\n    def __init__(self, in_channels, out_channels, kernel_size, stride=1,\n                 padding=0, dilation=1, groups=1, dense_conv=False,\n                 bias=True, padding_mode='zeros', use_batchnorm=True,\n                 pooling_type='max', pooling_kernel=1, pooling_stride=1,\n                 activation_type='relu', dropout_prob=0.5):\n"

In [101]:
Basenji = SeqNN(job)

('cnn0', BasenjiConvBlock(4, 312, 22, stride=1, padding=0, 
                                             dilation=1, groups=1, bias=False, 
                                             dense_conv=False, pooling_kernel=1, 
                                             pooling_stride=1, activation_type='relu', 
                                             dropout_prob=0.05))
('cnn1', BasenjiConvBlock(312, 368, 1, stride=1, padding=0, 
                                             dilation=1, groups=1, bias=False, 
                                             dense_conv=False, pooling_kernel=2, 
                                             pooling_stride=2, activation_type='relu', 
                                             dropout_prob=0.05))
('cnn2', BasenjiConvBlock(368, 435, 6, stride=1, padding=0, 
                                             dilation=1, groups=1, bias=False, 
                                             dense_conv=False, pooling_kernel=4, 
                          

In [121]:
for i in range(13):
    new_weights = torch.Tensor(check_weights['cnn{}/conv1d/kernel:0'.format(i)]).permute(2,1,0)
    new_gamma   = torch.Tensor(check_weights['cnn{}/batch_normalization/gamma:0'.format(i)])
    new_beta    = torch.Tensor(check_weights['cnn{}/batch_normalization/beta:0'.format(i)])
    new_mean    = torch.Tensor(check_weights['cnn{}/batch_normalization/moving_mean:0'.format(i)])
    new_var     = torch.Tensor(check_weights['cnn{}/batch_normalization/moving_variance:0'.format(i)])
    getattr( Basenji.network, 'cnn{}'.format(i) ).conv1d.weight.data = new_weights
    getattr( Basenji.network, 'cnn{}'.format(i) ).batch_normalization.weight.data = new_gamma
    getattr( Basenji.network, 'cnn{}'.format(i) ).batch_normalization.bias.data = new_beta
    getattr( Basenji.network, 'cnn{}'.format(i) ).batch_normalization.running_mean.data = new_mean
    getattr( Basenji.network, 'cnn{}'.format(i) ).batch_normalization.running_var.data = new_var
    getattr( Basenji.network, 'cnn{}'.format(i) ).batch_normalization.num_batches_tracked.data = torch.tensor(10000, dtype=torch.long)

new_weights = torch.Tensor(check_weights['final/dense/kernel:0']).permute(1,0).unsqueeze(2)
new_bias    = torch.Tensor(check_weights['final/dense/bias:0'])
getattr( Basenji.network, 'final' ).conv1d.weight.data = new_weights
getattr( Basenji.network, 'final' ).conv1d.bias.data = new_bias

In [122]:
getattr( Basenji.network, 'final' )

BasenjiConvBlock(
  (conv1d): Conv1d(1365, 4229, kernel_size=(1,), stride=(1,))
  (batch_normalization): Passthrough()
  (pooling): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (activation): ExpLinear()
  (dropout): Passthrough()
)

In [108]:
cnn12/conv1d/kernel:0 - (1, 1365, 1365)
cnn12/batch_normalization/gamma:0 - (1365,)
cnn12/batch_normalization/beta:0 - (1365,)
cnn12/batch_normalization/moving_mean:0 - (1365,)
cnn12/batch_normalization/moving_variance:0 - (1365,)
final/dense/kernel:0 - (1365, 4229)
final/dense/bias:0 - (4229,)



torch.Size([312, 4, 22])

In [ ]:
cnn7/conv1d/kernel:0 - (3, 825, 108)
cnn7/batch_normalization/gamma:0 - (108,)
cnn7/batch_normalization/beta:0 - (108,)
cnn7/batch_normalization/moving_mean:0 - (108,)
cnn7/batch_normalization/moving_variance:0 - (108,)

cnn12/conv1d/kernel:0 - (1, 1365, 1365)
cnn12/batch_normalization/gamma:0 - (1365,)
cnn12/batch_normalization/beta:0 - (1365,)
cnn12/batch_normalization/moving_mean:0 - (1365,)
cnn12/batch_normalization/moving_variance:0 - (1365,)
final/dense/kernel:0 - (1365, 4229)
final/dense/bias:0 - (4229,)


In [ ]:
class SeqNN(nn.Module):
    def __init__(self):
        super(SeqNN, self).__init__()
        ## Layer 1
        self.encoder_1 = nn.Sequential(OrderedDict([
          ('E1_1', Conv1dNorm(10, 64, 5, stride=1, padding=2, weight_norm=False)),
          ('E1_2', nn.ReLU()),
          ('E1_3', Conv1dNorm(64, 64, 5, stride=1, padding=2, weight_norm=False)),
          ('E1_4', nn.ReLU())
        ]))
        self.downsample_1 = nn.Sequential(OrderedDict([
          ('E1_5', nn.MaxPool1d(4, stride=4)),
          ('E1_6', Dropout1d(p=0.2))
        ]))
        self.encoder_2 = nn.Sequential(OrderedDict([
          ('E2_1', Conv1dNorm(64, 128, 5, stride=1, padding=2, weight_norm=False)),
          ('E2_2', nn.ReLU()),
          ('E2_3', Conv1dNorm(128, 128, 5, stride=1, padding=2, weight_norm=False)),
          ('E2_4', nn.ReLU())
        ]))
        self.downsample_2 = nn.Sequential(OrderedDict([
          ('E2_5', nn.MaxPool1d(4, stride=4)),
          ('E2_6', Dropout1d(p=0.2))
        ]))
        self.encoder_3 = nn.Sequential(OrderedDict([
          ('E3_1', Conv1dNorm(128, 256, 5, stride=1, padding=2, weight_norm=False)),
          ('E3_2', nn.ReLU()),
          ('E3_3', Conv1dNorm(256, 256, 5, stride=1, padding=2, weight_norm=False)),
          ('E3_4', nn.ReLU())
        ]))
        self.downsample_3 = nn.Sequential(OrderedDict([
          ('E3_5', nn.MaxPool1d(4, stride=4)),
          ('E3_6', Dropout1d(p=0.2))
        ]))
        self.encoder_4 = nn.Sequential(OrderedDict([
          ('E4_1', Conv1dNorm(256, 512, 5, stride=1, padding=2, weight_norm=False)),
          ('E4_2', nn.ReLU()),
          ('E4_3', Conv1dNorm(512, 512, 5, stride=1, padding=2, weight_norm=False)),
          ('E4_4', nn.ReLU())
        ]))
        self.decoder_1 = nn.Sequential(OrderedDict([
          ('D1_1', Decon1dNorm(512, 256, 8, stride=4, padding=2, weight_norm=False))
        ]))
        self.decoder_2 = nn.Sequential(OrderedDict([
          ('D2_1', Conv1dNorm((256+256), 256, 5, stride=1, padding=2, weight_norm=False)),
          ('D2_2', nn.ReLU()),
          ('D2_3', Conv1dNorm(256, 256, 5, stride=1, padding=2, batch_norm=False, weight_norm=False)),
          ('D2_4', nn.ReLU()),
          ('D2_5', Dropout1d(p=0.2)),
          ('D2_6', Decon1dNorm(256, 128, 8, stride=4, padding=2, weight_norm=False))
        ]))
        self.decoder_3 = nn.Sequential(OrderedDict([
          ('D3_1', Conv1dNorm((128+128), 128, 5, stride=1, padding=2, weight_norm=False)),
          ('D3_2', nn.ReLU()),
          ('D3_3', Conv1dNorm(128, 128, 5, stride=1, padding=2, batch_norm=False, weight_norm=False)),
          ('D3_4', nn.ReLU()),
          ('D3_5', Dropout1d(p=0.2)),
          ('D3_6', Decon1dNorm(128, 64, 8, stride=4, padding=2, weight_norm=False))
        ]))
        self.classifier = nn.Sequential(OrderedDict([
          ('C1_1', Conv1dNorm((64+64), 64, 5, stride=1, padding=2, weight_norm=False)),
          ('C1_2', nn.ReLU()),
          ('C1_3', Conv1dNorm(64, 64, 5, stride=1, padding=2, weight_norm=False)),
          ('C1_4', nn.ReLU()),
          ('C1_5', Conv1dNorm(64, 1, 1, stride=1, batch_norm=False, weight_norm=False))
        ]))
        self.auxiliary = nn.Sequential(OrderedDict([
          ('A1_1', Conv1dNorm(512, 512, 5, stride=1, padding=2, weight_norm=False)),
          ('A1_1b', nn.ReLU()),
          ('A1_2', Dropout1d(p=0.2)),
          ('A1_3', Conv1dNorm(512, 1, 1, stride=1, batch_norm=False, weight_norm=False))
        ]))
    def forward(self, x):
        # Layer 1
        layer1_hook = self.encoder_1( x[:,1:11,:] )
        main_hook   = self.downsample_1( layer1_hook )
        layer2_hook = self.encoder_2( main_hook )
        main_hook   = self.downsample_2( layer2_hook )
        layer3_hook = self.encoder_3( main_hook )
        main_hook   = self.downsample_3( layer3_hook )
        main_hook   = self.encoder_4( main_hook )
        auxi_hook   = self.auxiliary( main_hook )
        main_hook   = self.decoder_1( main_hook )
        main_hook   = torch.cat([main_hook,layer3_hook],dim=1)
        main_hook   = self.decoder_2( main_hook )
        main_hook   = torch.cat([main_hook,layer2_hook],dim=1)
        main_hook   = self.decoder_3( main_hook )
        main_hook   = torch.cat([main_hook,layer1_hook],dim=1)
        main_hook   = self.classifier( main_hook )
        return main_hook, auxi_hook
